In [2]:
import cv2
import os
import numpy as np
from helper_functions import extract_patches

In [2]:
hwsize = 32

# Arrays of rotations to be used for patch creation. 
pos_rots = [0, 45, 90]
neg_rots = [0, 45]

# Lists for the datapoints with no centers and no blue ratio data respectively
no_centers = []
no_br = []

# Dictionaries where the labels for each patch will be stored. Needed for training and testing
dict_labels_pos = {}
dict_labels_neg = {}

# Original dataset location
orig_dataset_dir = '/home/advo/PycharmProjects/ML_ND3_CapstoneProject/Dataset/mitosis'
patches_dataset_dir = '/home/advo/PycharmProjects/ML_ND3_CapstoneProject/Dataset_small/patches/'
os.chdir(orig_dataset_dir)
root = os.path.splitext(orig_dataset_dir)[0]
print(root)

/home/advo/PycharmProjects/ML_ND3_CapstoneProject/Dataset_small/09_10_11_12/


## !! Caution: Takes a long time to execute

In [4]:
for file in os.listdir(orig_dataset_dir):

    if file.endswith(".tif"):
        
        # print(file)
        print("ram usage: " + str(psutil.virtual_memory().percent))
        basename = os.path.splitext(file)[0]
        filename = os.path.join(root + file)
        no_gt = False
        
                
        image = cv2.imread(file, cv2.IMREAD_UNCHANGED)
        
        if type(image) == type(None):
        
            print("Error: file " + file + " not read")
        
        else:
            
            # Resize the image to 20x magnification (initially it was 2000x2000)
            # The reason behind this is to ensure that the entire mitotic center is contained in a patch
            image = cv2.resize(src=image, dsize=(1000,1000))
            
            # Form the name of the Ground Truth file
            # Will later check if it actually exists
            gt_maskname = os.path.join(root + basename + "_pc.png")

            # Read the Ground Truth mask
            gt_mask = cv2.imread(gt_maskname, cv2.IMREAD_UNCHANGED)

            # Check if the image has a ground truth associated
            if type(gt_mask) is type(None):

                # Add to a list of missing ground truths
                no_centers.append(basename)
                print("no GT " + os.path.split(gt_maskname)[1])
                no_gt = True

            else:

                # Resize the image to 20x magnification
                # The reason behind this is to ensure that the entire mitotic center is contained in a patch
                gt_mask = cv2.resize(src=gt_mask, dsize=(1000,1000))
                
                # Extract the coordinates of the centers 
                gt_result = np.where(gt_mask == 255)
                # print("number of GT pos pix in " + gt_maskname + "= " + str(len(gt_result[0])))
                
                # Extract the patches surrounding the positive pixels
                extract_patches(image, gt_result, 'class1', pos_rots, hwsize, dict_labels_pos, patches_dataset_dir, basename)



            # Form the name of the Blue Ratio Segmentation mask file
            br_maskname = os.path.join(root + basename + "_cmask.png")

            # Read the Blue Ratio Segmentation mask
            br_mask = cv2.imread(br_maskname, cv2.IMREAD_UNCHANGED)

            # Check if the image has a BR mask associated
            if type(br_mask) == type(None):

                # Add to a list of missing BR masks
                no_br.append(basename)
                print("no BR " + str(br_maskname))

            else:

                # Resize the image to 20x magnification
                # The reason behind this is to ensure that the entire mitotic center is contained in a patch
                br_mask = cv2.resize(src=br_mask, dsize=(1000,1000))        
        
                # Extract the coordinates of the centers 
                br_result = np.where(br_mask == 255)
                # print("Size of BR mask before removing pos pix: " + str(len(br_result[0])) + ", " +
                # str(len(br_result[1])))

                # Check if the Ground Truth mask exists 
                # (ie. if there are any positive centers in the current image)
                if not no_gt:

                    # Remove the positive centers from the BR mask
                    for id1, id2 in zip(gt_result[0], gt_result[1]):
                        br_mask[id1][id2] = 0

                    # Extract the coordinates of the centers 
                    br_result = np.where(br_mask == 255)
                    # print("Size of BR mask after removing pos pix: " + str(len(br_result[0])) + ", " +
                    # str(len(br_result[1])))

                # Extract the patches whether we had positive centers or not
                # These ones will be used as negative samples
                extract_patches(image, br_result, 'class0', neg_rots, hwsize, dict_labels_neg, patches_dataset_dir, basename)

         
    # break "file" loop
    # break
    
print("\nThe following "+ str(len(no_centers)) +" datapoints contain no Ground Truth info: ")
print(no_centers)

print("\nThe following "+ str(len(no_br)) +" datapoints contain no Blue Ration Segmentation info: ")
print(no_br)
            
# Free up the space
no_centers = None
no_br = None

ram usage: 82.3
no GT 09_10_pc.png
class0 number of pix after edge filtering= 162803
 final number of pix to be extracted:  326
ram usage: 82.3
no GT 10_02_pc.png
class0 number of pix after edge filtering= 62489
 final number of pix to be extracted:  125
ram usage: 82.5
class1 number of pix after edge filtering= 67
 final number of pix to be extracted:  67
class0 number of pix after edge filtering= 259659
 final number of pix to be extracted:  519
ram usage: 82.4
class1 number of pix after edge filtering= 134
 final number of pix to be extracted:  134
class0 number of pix after edge filtering= 166165
 final number of pix to be extracted:  332
ram usage: 81.8
class1 number of pix after edge filtering= 0
 final number of pix to be extracted:  0
class0 number of pix after edge filtering= 158445
 final number of pix to be extracted:  317
ram usage: 82.1
no GT 09_01_pc.png
class0 number of pix after edge filtering= 149671
 final number of pix to be extracted:  299
ram usage: 82.4
class1 num

**Load an image and visualise/resize with cv2 (for testing purposes)**

In [6]:
mask = cv2.imread('/home/advo/PycharmProjects/ML_ND3_CapstoneProject/Dataset/mitosis/01_01_cmask.png', cv2.IMREAD_UNCHANGED)
print(mask.shape)

(2000, 2000)


In [ ]:
cv2.imshow("image", mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
image = cv2.resize(src=image, dsize=(1000,1000))
cv2.imshow("image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()